# UCI credit card data from [Kaggle](https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset)

### Default Prediction using Keras. Original template from Wilson Kan. 

### Other references:

deepcredit/default-prediction.ipynb

[Ranking no. 1 on Kaggle for Predicting Consumer Debt Default ]( https://nycdatascience.com/blog/student-works/kaggle-predict-consumer-credit-default/) 

In [1]:
from IPython.display import display
import pandas as pd

pd.set_option('max_columns', 55)
data = pd.read_csv("defaultcreditcard.csv", skiprows = 1)

# display(data.describe())
# display(data.head(5))
# data.shape

### Create indicator variables for the categorical variables

In [2]:
import numpy as np
data['MALE'] = np.where(data['SEX']==1, 1, 0)
data['FEMALE'] = np.where(data['SEX']==2, 1, 0)
data['MARRIED'] = np.where(data['MARRIAGE']==1, 1, 0)
data['SINGLE'] = np.where(data['MARRIAGE']==2, 1, 0)
data['STATUS_OTHER'] = np.where(np.logical_or(data['MARRIAGE']==3,data['MARRIAGE']==0), 1, 0)
data['GRADUATE'] = np.where(data['EDUCATION']==1, 1, 0)
data['UNIVERSITY'] = np.where(data['EDUCATION']==2, 1, 0)
data['HIGHSCHOOL'] = np.where(data['EDUCATION']==3, 1, 0)
data['EDU_OTHER'] = np.where(np.logical_or(data['EDUCATION']>=4,data['EDUCATION']==0), 1, 0)

# Normalizing the data for age and others (making use of the map and lambda function to change the data)
def my_normalize(s):
    P90 = data[s].quantile(0.90)
    data[s+"_NORM"] = data[s].map(lambda x: max(min(x / P90, 1), 0))
def pay_normalize(s):
    data[s+"_NORM"] = data[s].map(lambda x: x / 9)


AGEMEAN = data['AGE'].mean()
AGESTD = data['AGE'].std()

header = list(data.columns.values)
my_normalize(header[1])
for s in header[12:24]:
    my_normalize(s)
for s in header[6:12]:
    pay_normalize(s)
data['AGE_NORM'] = data['AGE'].map(lambda x: (x - AGEMEAN) / AGESTD)


# data.head(5)
# data.describe()

### Form a new data set with the normalized variables and split it into 70% training, 30% testing

In [3]:
from sklearn.model_selection import train_test_split

header = list(data.columns.values)
# use only the normalizeddata from column 25 and beyond 
model_data_full = data[header[24:]]

header = list(model_data_full.columns.values)

# X_train, X_test, y_train, y_test = train_test_split(model_data_full[header[1:]], model_data_full[header[0]], test_size=0.3, random_state=2016)
X_train, X_test = train_test_split(model_data_full[header[1:]], test_size=0.3, random_state=2016)
# use a different method "iloc" to extract the sub dataframe so that it remains a n x 1 matrix instead of a series
y_train, y_test = train_test_split(model_data_full.iloc[:,[0]], test_size=0.3, random_state=2016)



In [13]:
# Convert the data type to numpy array from of data frame
# display(type(X_train))
# display(type(y_train))
# display(X_train.shape)
# display(y_train.shape)
X_train = X_train.as_matrix()
y_train = y_train.as_matrix()
X_test = X_test.as_matrix()
y_test = y_test.as_matrix()

# np.ravel(y_train)
display(type(X_train))
display(type(y_train))
display(type(X_test))
display(type(y_test))


AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [5]:
import keras

C:\Users\yan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Import Keras using Tensorflow Backend

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics

In [18]:
model = Sequential()
model.add(Dense(64, input_dim=29, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
              # metrics=['accuracy', 'mae', 'mape'])

In [19]:
model.fit(X_train, y_train,
          epochs=20,
          batch_size=128)

Epoch 1/20
21000/21000 [==============================] - 1s 32us/step - loss: 0.4817 - acc: 0.7881
Epoch 2/20
21000/21000 [==============================] - 0s 14us/step - loss: 0.4554 - acc: 0.8041
Epoch 3/20
21000/21000 [==============================] - 0s 13us/step - loss: 0.4485 - acc: 0.8072
Epoch 4/20
21000/21000 [==============================] - 0s 13us/step - loss: 0.4447 - acc: 0.8105
Epoch 5/20
21000/21000 [==============================] - 0s 12us/step - loss: 0.4423 - acc: 0.8113
Epoch 6/20
21000/21000 [==============================] - 0s 13us/step - loss: 0.4403 - acc: 0.8127
Epoch 7/20
21000/21000 [==============================] - 0s 14us/step - loss: 0.4388 - acc: 0.8143
Epoch 8/20
21000/21000 [==============================] - 0s 13us/step - loss: 0.4370 - acc: 0.8164
Epoch 9/20
21000/21000 [==============================] - 0s 13us/step - loss: 0.4349 - acc: 0.8161
Epoch 10/20
21000/21000 [==============================] - 0s 14us/step - loss: 0.4338 - acc: 0.8182

In [23]:
score = model.evaluate(X_test, y_test, verbose =1 )

print("\n")
print("test accuracy: ")
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print("\n")      
# print("%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
# print("\n")      
# print("%s: %.2f%%" % (model.metrics_names[3], score[3]*100))

# cvscores.append(scores[1] * 100)
# print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

#How to do AUC measure under keras?
# https://github.com/fchollet/keras/issues/832


9000/9000 [==============================] - 0s 21us/step


test accuracy: 
acc: 81.76%


